In [31]:
from datasets import load_metric, load_from_disk, Sequence, Value, Features, Dataset, DatasetDict
f = Features(
            {
                "context": Value(dtype="string", id=None),
                "id": Value(dtype="string", id=None),
                "question": Value(dtype="string", id=None),
            }
        )

f

{'context': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None)}

In [32]:
df = pd.read_csv('ES_contest_main.csv')
dataset = Dataset.from_pandas(df, features=f)

In [1]:
# !apt-get update && apt-get install -y gnupg2
# !wget -qO - https://artifacts.elastic.co/GPG-KEY-elasticsearch | apt-key add -
# !apt-get install apt-transport-https
# !echo "deb https://artifacts.elastic.co/packages/7.x/apt stable main" | tee /etc/apt/sources.list.d/elastic-7.x.lis
# !apt-get update && apt-get install elasticsearch

# # nori tokenizer 설치
# !service elasticsearch start
# !cd /usr/share/elasticsearch/
# !bin/elasticsearch-plugin install analysis-nori
# !service elasticsearch restart

# # pip
# !pip install elasticsearch

In [2]:
# !python -m pip install elasticsearch
# !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
# !tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
# !chown -R daemon:daemon elasticsearch-7.9.2

In [3]:
# import pandas as pd
# import json

# with open('/opt/ml/data/wikipedia_documents.json', 'r') as f:
#         wiki_data = pd.DataFrame(json.load(f)).transpose()


In [4]:
# wiki_data

In [5]:
# with open('/opt/ml/data/wikipedia_documents.json', "r") as f:
#         wiki = json.load(f)

# wiki

In [6]:
# path_to_elastic = '/opt/ml/code/elasticsearch-7.9.2/bin/elasticsearch'
#     es_server = Popen([path_to_elastic],
#                     stdout=PIPE, stderr=STDOUT,
#                     preexec_fn=lambda: os.setuid(1)  # as daemon
#                     )
#     !sleep(30)

#     config = {'host':'localhost', 'port':9200}
#     es = Elasticsearch([config])

In [7]:
# import os
# from subprocess import Popen, PIPE, STDOUT
# es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
#                    stdout=PIPE, stderr=STDOUT,
#                    preexec_fn=lambda: os.setuid(1)  # as daemon
#                   )
# # wait until ES has started
# ! sleep 30

In [8]:
# ! /opt/ml/code/elasticsearch-7.9.2/bin/elasticsearch-plugin install analysis-nori

In [9]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [10]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [11]:
es.info()

{'name': '75f1965bd0f6',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '-Dt_W5UGTiCA0uuyzfGyKA',
 'version': {'number': '7.9.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e',
  'build_date': '2020-09-23T00:45:33.626720Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [12]:
es.indices.get('document')

{'document': {'aliases': {},
  'mappings': {'properties': {'text': {'type': 'text',
     'analyzer': 'my_analyzer',
     'similarity': 'my_similarity'},
    'title': {'type': 'text',
     'analyzer': 'my_analyzer',
     'similarity': 'my_similarity'}}},
  'settings': {'index': {'number_of_shards': '1',
    'provided_name': 'document',
    'similarity': {'my_similarity': {'type': 'BM25'}},
    'creation_date': '1635928325121',
    'analysis': {'filter': {'my_shingle_f': {'type': 'shingle'}},
     'analyzer': {'my_analyzer': {'filter': ['lowercase',
        'my_shingle_f',
        'nori_readingform',
        'nori_number'],
       'decompound_mode': 'mixed',
       'type': 'custom',
       'stopwords': '_korean_',
       'tokenizer': 'nori_tokenizer'}}},
    'number_of_replicas': '1',
    'uuid': 'N99HqzlVSPCqQUpOHtVO1A',
    'version': {'created': '7090299'}}}}}

In [13]:
import json
import pandas as pd

with open('/opt/ml/data/wikipedia_documents.json', 'r') as f:
    wiki_data = pd.DataFrame(json.load(f)).transpose()

In [14]:
es.indices.delete(index="document", ignore=[400, 404])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [15]:
from tqdm import tqdm
mapping = {
                      'settings':{
                          'analysis':{
                              'analyzer':{
                                  'my_analyzer':{
                                      "type": "custom",
                                      'tokenizer':'nori_tokenizer',
                                      'decompound_mode':'mixed',
                                      'stopwords':'_korean_',
                                      "filter": ["lowercase",
                                                 "my_shingle_f",
                                                 "nori_readingform",
                                                 "nori_number"]
                                  }
                              },
                              'filter':{
                                  'my_shingle_f':{
                                      "type": "shingle"
                                  }
                              }
                          },
                          'similarity':{
                              'my_similarity':{
                                  'type':'BM25',
                              }
                          }
                      },
                      'mappings':{
                          'properties':{
                              'title':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity'
                              },
                              'text':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity'
                              }
                          }
                      }
                  }

In [16]:
import os

# os.environ.setdefault("DJANGO_SETTINGS_MODULE", "code.settings")



In [17]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

In [18]:
# from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document", custom_mapping=mapping)

import json
with open('/opt/ml/data/wikipedia_documents.json', "r") as f:
    wiki = json.load(f)
contexts = list(dict.fromkeys([v['text'] for v in wiki.values()]))


dicts = [
    {
        'text': context,
    } for context in tqdm(contexts)
]
document_store.write_documents(dicts)

11/03/2021 08:34:21 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.002s]
11/03/2021 08:34:21 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.089s]
11/03/2021 08:34:21 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.001s]
100%|██████████| 56737/56737 [00:00<00:00, 1017149.20it/s]
11/03/2021 08:34:25 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.464s]
11/03/2021 08:34:26 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.181s]
11/03/2021 08:34:28 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.172s]
11/03/2021 08:34:29 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.118s]
11/03/2021 08:34:30 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.1

KeyboardInterrupt: 

In [ ]:
document_store

In [ ]:
dicts

In [ ]:
#only retriever
from haystack.retriever import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store)
from haystack.pipeline import DocumentSearchPipeline
pipe = DocumentSearchPipeline(retriever)

In [ ]:
# #reader(FARM of Transformer) +retriever
# from haystack.reader.farm import FARMReader
# from haystack.reader.transformers import TransformersReader
# reader = TransformersReader(model_name_or_path="/opt/ml/code/models/koelectra_test",max_seq_len=300, doc_stride=120)

# from haystack.pipeline import ExtractiveQAPipeline
# pipe = ExtractiveQAPipelinee(reader,retriever)

In [ ]:
from datasets import load_from_disk
testset=load_from_disk('/opt/ml/data/test_dataset')
testset=testset['validation']

In [ ]:
question=testset[0]['question']
prediction = pipe.run(query=question, params={"retriever": {"top_k": 30}})
prediction['documents'][:5]

In [ ]:
# #dictionary로 top1 문서내용을 json으로 저장
# result={}
# #for i in range(10): #len(testset['question'])
# for example in testset:
#     question=example["question"]
#     prediction = pipe.run(query=question, params={"retriever": {"top_k": 10}})
#     #from haystack.utils import print_answers
#     if prediction['d'][0]['probability']>0.99:
#         result[example["id"]]=prediction['answers'][0]['answer']
#     else:
#         result[example["id"]]=None

# with open('elastic_reader.json', "w") as writer:
#     writer.write(json.dumps(result, indent=4, ensure_ascii=False) + "\n")

In [ ]:
# question = testset[0]['question']
# top_k_docs = pipe.run(question, params={"retriever": {"top_k": 10}})

In [ ]:
# #top1 score json으로 저장
# save_score={}

# for idx, example in enumerate(tqdm(testset, desc="elasticsearch: ")):
#     # relev_doc_ids = [el for i, el in enumerate(self.ids) if i in doc_indices[idx]]
#     question=example["question"]
#     top_k_docs = pipe.run(question, params={"retriever": {"top_k": 10}})

#     query = {
#         'query':{
#             'bool':{
#                 'must':[
#                           {'match':{'text':question}}
#                 ],
#                 'should':[
#                           {'match':{'text':question}}
#                 ]
#             }
#         }
#     }
#     doc = es.search(index='document',body=query,size=30)['hits']['hits']

    
#     save_score[example['id']]=doc[0]['_score']
    
# with open('top1_score.json', "w") as writer:
#     writer.write(json.dumps(save_score, indent=4, ensure_ascii=False) + "\n")

In [ ]:
docs = []
quries = []

In [ ]:
testset[0]

In [ ]:
#datafram을 pickle로 top5 저장
import pandas as pd
total = []
docs = []
quries = []
for idx, example in enumerate(tqdm(testset, desc="elasticsearch: ")):
    # relev_doc_ids = [el for i, el in enumerate(self.ids) if i in doc_indices[idx]]
    question=example["question"]
    top_k_docs = pipe.run(question, params={"retriever": {"top_k": 30}})

    # query = {
    #     'query':{
    #         'bool':{
    #             'must':[
    #                       {'match':{'text':question}}
    #             ],
    #             'should':[
    #                       {'match':{'text':question}}
    #             ]
    #         }
    #     }
    # }

    query = {
            'query': {
                'match': {
                    'text': question
                    }
                }
            }
    quries.append(query)
    
    doc = es.search(index='document',body=query,size=30)['hits']['hits']
    cc = ''
    docs.append(doc)
    
    for i in range(10):
        cc += doc[i]['_source']['text']
        

    tmp = {
        "question": example["question"],
        "id": example['id'],
        "context_id": doc[0]['_id'],  # retrieved id
        "context": cc # doc[0]['_source']['text']+doc[1]['_source']['text']+doc[2]['_source']['text']+doc[3]['_source']['text']+doc[4]['_source']['text'] # retrieved doument
    }
 
    if 'context' in example.keys() and 'answers' in example.keys():
        tmp["original_context"] = example['context']  # original document
        tmp["answers"] = example['answers']           # original answer
    total.append(tmp)

In [ ]:
# docs
query

In [ ]:
docs

In [ ]:
cqas = pd.DataFrame(total)
# cqas = pd.read_pickle("elastic.pkl")
cqas

# ---------------RUN MRC TEST--------------------

In [ ]:
df = cqas
df['context'][0]

In [ ]:
from datasets import (
    load_metric,
    load_from_disk,
    Sequence,
    Value,
    Features,
    Dataset,
    DatasetDict,
)

In [ ]:
f = Features(
    {
        "context": Value(dtype="string", id=None),
        "id": Value(dtype="string", id=None),
        "question": Value(dtype="string", id=None),
    }
)

In [ ]:
from inference import main, run_mrc

In [ ]:
datasets = DatasetDict({"validation": Dataset.from_pandas(df, features=f)})
datasets

In [ ]:
"""
Open-Domain Question Answering 을 수행하는 inference 코드 입니다.
대부분의 로직은 train.py 와 비슷하나 retrieval, predict 부분이 추가되어 있습니다.
"""

import os
import logging
import sys
from typing import Callable, List, Dict, NoReturn, Tuple

import numpy as np

from datasets import (
    load_metric,
    load_from_disk,
    Sequence,
    Value,
    Features,
    Dataset,
    DatasetDict,
)

from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer, AutoModel

from transformers import (
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

from utils_qa import postprocess_qa_predictions, check_no_error
from trainer_qa import QuestionAnsweringTrainer
from retrieval import SparseRetrieval
from train_dpr import DenseRetrieval

from arguments import (
    ModelArguments,
    DataTrainingArguments,
)

from bm25_ensemble import BM25EnsembleRetrieval

logger = logging.getLogger(__name__)

In [ ]:
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, TrainingArguments)
)
model_args, data_args, training_args = parser.parse_args_into_dataclasses()
# 인스턴스 클래스로 감싸주는 거 하면 좋을 듯!
training_args.do_train = True

print(f"model is from {model_args.model_name_or_path}")
print(f"data is from {data_args.dataset_name}")

# logging 설정
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

# verbosity 설정 : Transformers logger의 정보로 사용합니다 (on main process only)
logger.info("Training/evaluation parameters %s", training_args)

# 모델을 초기화하기 전에 난수를 고정합니다.
set_seed(training_args.seed)

datasets = load_from_disk("../data/train_dataset")
print(datasets)

# AutoConfig를 이용하여 pretrained model 과 tokenizer를 불러옵니다.
# argument로 원하는 모델 이름을 설정하면 옵션을 바꿀 수 있습니다.
config = AutoConfig.from_pretrained(
    model_args.config_name
    if model_args.config_name
    else model_args.model_name_or_path,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name
    if model_args.tokenizer_name
    else model_args.model_name_or_path,
    use_fast=True,
)
model = AutoModelForQuestionAnswering.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
)
# True일 경우 : run passage retrieval
# eval_retrieval의 default는? true

In [ ]:
run_mrc(data_args, training_args, model_args, datasets, tokenizer, model)

In [ ]:
def run_mrc(
    data_args: DataTrainingArguments,
    training_args: TrainingArguments,
    model_args: ModelArguments,
    datasets: DatasetDict,
    tokenizer,
    model,
) -> NoReturn:

    # eval 혹은 prediction에서만 사용함
    column_names = datasets["validation"].column_names

    question_column_name = "question" if "question" in column_names else column_names[0]
    context_column_name = "context" if "context" in column_names else column_names[1]
    answer_column_name = "answers" if "answers" in column_names else column_names[2]

    # Padding에 대한 옵션을 설정합니다.
    # (question|context) 혹은 (context|question)로 세팅 가능합니다.
    pad_on_right = tokenizer.padding_side == "right"

    # 오류가 있는지 확인합니다.
    last_checkpoint, max_seq_length = check_no_error(
        data_args, training_args, datasets, tokenizer
    )

    # Validation preprocessing / 전처리를 진행합니다.
    # start, end 정보가 없음, 만들 수가 없다.
    def prepare_validation_features(examples):
        # truncation과 padding(length가 짧을때만)을 통해 toknization을 진행하며, stride를 이용하여 overflow를 유지합니다.
        # 각 example들은 이전의 context와 조금씩 겹치게됩니다.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=data_args.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            # return_token_type_ids=False, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
            padding="max_length" if data_args.pad_to_max_length else False,
        )

        # 길이가 긴 context가 등장할 경우 truncate를 진행해야하므로, 해당 데이터셋을 찾을 수 있도록 mapping 가능한 값이 필요합니다.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        # evaluation을 위해, prediction을 context의 substring으로 변환해야합니다.
        # corresponding example_id를 유지하고 offset mappings을 저장해야합니다.
        tokenized_examples["example_id"] = []

        for i in range(len(tokenized_examples["input_ids"])):
            # sequence id를 설정합니다 (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0

            # 하나의 example이 여러개의 span을 가질 수 있습니다.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])

            # context의 일부가 아닌 offset_mapping을 None으로 설정하여 토큰 위치가 컨텍스트의 일부인지 여부를 쉽게 판별할 수 있습니다.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]
        return tokenized_examples

    eval_dataset = datasets["validation"]

    # Validation Feature 생성
    eval_dataset = eval_dataset.map(
        prepare_validation_features,
        batched=True,
        num_proc=data_args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not data_args.overwrite_cache,
    )

    # Data collator
    # flag가 True이면 이미 max length로 padding된 상태입니다.
    # 그렇지 않다면 data collator에서 padding을 진행해야합니다.
    data_collator = DataCollatorWithPadding(
        tokenizer, pad_to_multiple_of=8 if training_args.fp16 else None
    )

    # Post-processing:
    def post_processing_function(
        examples,
        features,
        predictions: Tuple[np.ndarray, np.ndarray],
        training_args: TrainingArguments,
    ) -> EvalPrediction:
        # Post-processing: start logits과 end logits을 original context의 정답과 match시킵니다.
        predictions = postprocess_qa_predictions(
            examples=examples,
            features=features,
            predictions=predictions,
            max_answer_length=data_args.max_answer_length,
            output_dir=training_args.output_dir,
        )
        # Metric을 구할 수 있도록 Format을 맞춰줍니다.
        formatted_predictions = [
            {"id": k, "prediction_text": v} for k, v in predictions.items()
        ]

        if training_args.do_predict:
            return formatted_predictions
        elif training_args.do_eval:
            references = [
                {"id": ex["id"], "answers": ex[answer_column_name]}
                for ex in datasets["validation"]
            ]

            return EvalPrediction(
                predictions=formatted_predictions, label_ids=references
            )

    metric = load_metric("squad")

    def compute_metrics(p: EvalPrediction) -> Dict:
        return metric.compute(predictions=p.predictions, references=p.label_ids)

    print("init trainer...")
    # Trainer 초기화
    trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=None,
        eval_dataset=eval_dataset,
        eval_examples=datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
    )

    logger.info("*** Evaluate ***")

    #### eval dataset & eval example - predictions.json 생성됨
    if training_args.do_predict:
        predictions = trainer.predict(
            test_dataset=eval_dataset, test_examples=datasets["validation"]
        )

        # predictions.json 은 postprocess_qa_predictions() 호출시 이미 저장됩니다.
        print(
            "No metric can be presented because there is no correct answer given. Job done!"
        )

    if training_args.do_eval:
        metrics = trainer.evaluate()
        metrics["eval_samples"] = len(eval_dataset)

        trainer.log_metrics("test", metrics)
        trainer.save_metrics("test", metrics)

In [ ]:
run_mrc(data_args, training_args, model_args, datasets, tokenizer, model)

In [ ]:
import argparse
import json
import os
import time
from subprocess import Popen, PIPE, STDOUT

from datasets import load_from_disk
from elasticsearch import Elasticsearch
# from prepare_dataset import make_custom_dataset # ?
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [ ]:
path_to_elastic = '/opt/ml/code/elasticsearch-7.9.2/bin/elasticsearch'
es_server = Popen([path_to_elastic],
                stdout=PIPE, stderr=STDOUT,
                preexec_fn=lambda: os.setuid(1)  # as daemon
                )
!sleep 30

config = {'host':'localhost', 'port':9200}
es = Elasticsearch([config])

index_config = {
    "settings": {
        "analysis": {
            "filter":{
                "my_stop_filter": {
                    "type" : "stop",
                    "stopwords_path" : "user_dic/my_stop_dic.txt"
                }
            },
            "analyzer": {
                "nori_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "decompound_mode": "mixed",
                    "filter" : ["my_stop_filter"]
                }
            }
        },
        'similarity':{
            'my_similarity':{
                'type':'BM25',
            }
        }
    },
    "mappings": {
        "dynamic": "strict", 
        "properties": {
            "document_text": {"type": "text", "analyzer": "nori_analyzer"}
            }
        }
    }

print('elastic serach ping :', es.ping())
print(es.indices.create(index='wiki-index', body=index_config, ignore=400))

In [ ]:
# if not os.path.isfile("../data/preprocess_train.pkl") :
    #     make_custom_dataset("../data/preprocess_train.pkl")
#     train_file = load_from_disk("../data/train_dataset")["train"]
#     validation_file = load_from_disk("../data/train_dataset")["validation"]
train_file = load_from_disk("../data/train_dataset")["train"]
validation_file = load_from_disk("../data/train_dataset")["validation"]

#[wiki-index, wiki-index-split-400, wiki-index-split-800, wiki-index-split-1000]

dataset_path = "../data/wikipedia_documents.json"
    
# if not os.path.isfile(dataset_path) :
#     print(dataset_path)
#     make_custom_dataset(dataset_path)

with open(dataset_path, "r") as f:
    wiki = json.load(f)
wiki_contexts = list(dict.fromkeys([v['text'] for v in wiki.values()]))

qa_records = [{"example_id" : train_file[i]["id"], "document_title" : train_file[i]["title"], "question_text" : train_file[i]["question"], "answer" : train_file[i]["answers"]} for i in range(len(train_file))]
wiki_articles = [{"document_text" : wiki_contexts[i]} for i in range(len(wiki_contexts))]

In [ ]:
# qa_records

In [ ]:
# wiki_articles

In [ ]:
import time

def set_datas() :
    # if not os.path.isfile("../data/preprocess_train.pkl") :
    #     make_custom_dataset("../data/preprocess_train.pkl")
#     train_file = load_from_disk("../data/train_dataset")["train"]
#     validation_file = load_from_disk("../data/train_dataset")["validation"]
    train_file = load_from_disk("../data/train_dataset")["train"]
    validation_file = load_from_disk("../data/train_dataset")["validation"]

    #[wiki-index, wiki-index-split-400, wiki-index-split-800, wiki-index-split-1000]
    
    dataset_path = "../data/wikipedia_documents.json"
        
    # if not os.path.isfile(dataset_path) :
    #     print(dataset_path)
    #     make_custom_dataset(dataset_path)

    with open(dataset_path, "r") as f:
        wiki = json.load(f)
    wiki_contexts = list(dict.fromkeys([v['text'] for v in wiki.values()]))

    qa_records = [{"example_id" : train_file[i]["id"], "document_title" : train_file[i]["title"], "question_text" : train_file[i]["question"], "answer" : train_file[i]["answers"]} for i in range(len(train_file))]
    wiki_articles = [{"document_text" : wiki_contexts[i]} for i in range(len(wiki_contexts))]
    return qa_records, wiki_articles


def set_index_and_server() :
    path_to_elastic = '/opt/ml/code/elasticsearch-7.9.2/bin/elasticsearch'
    es_server = Popen([path_to_elastic],
                    stdout=PIPE, stderr=STDOUT,
                    preexec_fn=lambda: os.setuid(1)  # as daemon
                    )
    time.sleep(30)

    config = {'host':'localhost', 'port':9200}
    es = Elasticsearch([config])

    index_config = {
        "settings": {
            "analysis": {
                "filter":{
                    "my_stop_filter": {
                        "type" : "stop",
                        "stopwords_path" : "user_dic/my_stop_dic.txt"
                    }
                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer",
                        "decompound_mode": "mixed",
                        "filter" : ["my_stop_filter"]
                    }
                }
            },
            'similarity':{
                'my_similarity':{
                    'type':'BM25',
                }
            }
        },
        "mappings": {
            "dynamic": "strict", 
            "properties": {
                "document_text": {"type": "text", "analyzer": "nori_analyzer"}
                }
            }
        }

    print('elastic serach ping :', es.ping())
    print(es.indices.create(index='wiki-index', body=index_config, ignore=400))
    return es


In [ ]:
_, wiki_articles = set_datas()
es = set_index_and_server()
es

In [ ]:
def populate_index(es_obj, index_name, evidence_corpus):

    for i, rec in enumerate(tqdm(evidence_corpus)):
        try:
            index_status = es_obj.index(index=index_name, id=i, body=rec)
        except:
            print(f'Unable to load document {i}.')
            
    n_records = es_obj.count(index=index_name)['count']
    print(f'Succesfully loaded {n_records} into {index_name}')